# Warp pRF result volumes to NMT v1.2    
This is useful for connective field analysis. pRF data are analyzed in the individual animals' functional spaces, but other pipelines may want them in NMT space. This workflow warps existing pRF-data to the new space. (NB! note that warping to NMT is now part of the pre-processing pipeline, but pRF fits were done in native space)

This notebook assumes that:     
- `${BasePath}/Results` is a folder or symlink with result files     
- `${{BasePath}/Reference` contains a copy of the `manual-masks` refernces and warps from `NHP-BIDS`

In [36]:
cd ..; BasePath=$(pwd); cd Notebooks

src_base=

trg_base=

warp_base=

### Configure     
Here, you should provide the information needed to do the warps    

In [1]:
# loop over monkeys

# loop over results



In [2]:
subj=eddy # subject; BIDS convention, do not capitalize
src_fld=${BasePath}/Results/Eddy-ctcheckerboard # source folder with results that need warping
trg_fld=${BasePath}/Results/Eddy-ctcheckerboard-nmt # target folder where warped files will go
srcspace='func' # the original space the results are in
trgspace='nmt' # the results we want them warped to 

This information should be automatically populated based on the information above.

In [38]:
warp_fld=${BasePath}/Reference/sub-${subj}/warps # folder where the warps are 
warp=${warp_fld}/sub-${subj}_${srcspace}2${trgspace}_res-1x1x1.mat # warp file for flirt 
ref=${warp_fld}/sub-${subj}_${srcspace}2${trgspace}_res-1x1x1.nii.gz # reference volume

We first make a copy of the source folder to create the target folder. Doing it this way ensures, we won't touch the original data and we can then do the warping with a very simple command using the `find` function to identify all nifti files. The cell is configured to only copy when the target folder does not exist

In [39]:
# copy the results folder so we can easily overwrite
if ! [ -d "${trg_fld}" ]; then
    cp -r ${src_fld} ${trg_fld};
else
    echo ' >> Target folder already exist. Are you sure this is what you want?'
    echo 'Will not automatically overwrite, but make sure you do not apply warps multiple times!'
fi

### Perform the warps    
The command below will warp all nifti files with the specified warp. Let's unpack the command a bit:    
`find` looks inside `${trg_fld}` for all files ending in `.nii.gz`. The `-execdir` flag means the function will then move focus to the folder where the found file is located and excute `flirt` to apply the warp `${warp}` to the found file with `${ref}` as the reference volume grid. It will overwrite the found file with the warped version of itself. `"{}"` are placeholders for the result of `find`.

This will of course take a while to finish. You can speed it up by restricting the search (e.g., to *zstat*.nii.gz) but keep in mind that in that case there will still be unwarped files in the target folder. Probably to wait a a bit and do the entire folder. 

In [40]:
# warp all the nifti files in the results folder
find "${trg_fld}" -name *.nii.gz -execdir \
    flirt -applyxfm -init ${warp} -in "{}" -ref ${ref} -out "{}" \;